In [ ]:
# Automatically set base path to the project directory where the notebook is running
from pathlib import Path

# This gets the directory where the current notebook is located
base_path = Path.cwd()

print(f"📂 Base path automatically set to: {base_path}")



<div class="alert alert-block alert-info">
<b>Note:</b> 

Copy the Basepath to ChatGPT memory.

</div>

# 📊 Data Conversion

## Purpose
Simplify files so that Python can read and parse data easily. Get scripts to process data into different filetypes, and number formats. Convert the time data to a date time that can synchronize a hydrograph input to a date time input.


# 🔢 Numerical Precision Float Operations

## Purpose
Precision is crucial in 2D modeling and large data operations because it affects the accuracy of calculations and the reliability of results. Inadequate precision can lead to errors that may impact analyses and simulations, making it vital to understand how to manage numerical data effectively.

## Examples
Here are some common operations involving float variables:

- **Calculate the area of every grid element in a mesh**:
- **Convert a float to an integer**:
- **Round a float to an integer**:

By mastering numerical precision and float variables, you can enhance the accuracy and reliability of your data analysis in various applications!

# 🤖ChatGPT Query Example

<div class="alert alert-block alert-info">
<b>GPT Text:</b>

- **Objective**: Create a python script that will print a table of 20 random float numbers with variable precision.
- **Data**: Define the difference between floats and integers with respect to python code.
- **Data Conversion**: Convert the list of variables to int, then round the first list to int and then convert both cols  back to float.
- **Output Format**: print output to cell output.
- **Dependencies**: Use Pandas.

</div>

## ▶️ Code Block
***The following cell is an example of what ChatGPT can build.  You can run this code with the dashboard controls or by "shift-enter"

In [ ]:
import pandas as pd
import numpy as np

# Set the random seed for reproducibility
np.random.seed(0)

# Create a list of float values
float_values = np.random.uniform(low=10.5, high=99.5, size=20)

# Create a DataFrame with the float values
df_floats = pd.DataFrame(float_values, columns=['Float Values'])

# Convert float values to integers directly (truncating the decimal part)
df_floats['Integer Values'] = df_floats['Float Values'].astype(int)

# Round float values before converting to integers
df_floats['Rounded Integers'] = df_floats['Float Values'].round().astype(int)

# Round float values to 4 decimal places
df_floats['Rounded to 4 Decimals'] = df_floats['Float Values'].round(4)

# Convert integer values back to floats
df_floats['Reconverted Floats'] = df_floats['Integer Values'].astype(float)

# Display the DataFrame with original float values, integer values, rounded integers, rounded to 4 decimals, and reconverted float values
df_floats



# 📐 Numerical Precision with Point Data

## Purpose
Precision is crucial in 2D modeling and large data operations because it directly influences the accuracy of spatial calculations and the reliability of results. For instance, calculating areas for grid elements in a mesh requires high precision to avoid small errors that can propagate through an analysis. This example demonstrates how numerical precision is handled when reading mesh data and calculating grid element areas.

## Examples
Here are some common operations involving float variables that you will explore in this notebook:

- **Calculate the area of each grid element in a mesh**: This operation demonstrates how spatial data (x, y coordinates) can be used to compute areas with precision.
  
By mastering precision in numerical data handling, you can enhance the accuracy of your floodplain modeling and grid analysis!


# 🤖 Example ChatGPT Query

<div class="alert alert-block alert-info">
<b>GPT Text:</b>

- **Objective**: Calculate the grid cell size and area for a centroid point file with space-delimited x, y, z data.
- **Data**: Each row in the file represents a centroid with x, y, and z coordinates. No headers are present.
- **Skip Lines**: Not applicable as there are no headers or initial lines to skip.
- **Output Format**: Save the output in a CSV file named `topoarea.csv` with columns for x, y, z, grid size, and area.
- **Dependencies**: Use Pandas to handle data reading, calculations, and writing to CSV.
- **File Paths**: Include variables to set file paths for reading the input (`topo.dat`) and writing the output (`topoarea.csv`).
- **Path**: Let Chat GPT know you want to join the base_path to this: \Data\Grids`
- **Request**: Can you create a Python script to calculate grid sizes based on consecutive y-values and compute areas assuming square cells, then write this data to a CSV file?

</div>

## ▶️ Code Block

The following code block is the result of this query. (Insert your Python code here if needed)


In [ ]:
import pandas as pd

# File paths
input_file_path = base_path / 'Data' / 'Grids' / 'topo.dat'
output_file_path = base_path / 'Data' / 'Grids' / 'topoarea.csv'

# Load data from space-delimited file without headers
data = pd.read_csv(input_file_path, header=None, sep=r'\s+', names=['x', 'y', 'z'])

# Calculate grid size assuming it's determined by the difference in consecutive y-values
data['grid_size'] = data['y'].diff().abs()

# Calculate the area of the square cell (grid size squared)
data['area'] = data['grid_size'] ** 2

# Handle the first row where the grid size calculation might result in a NaN
#data.fillna(method='bfill', inplace=True)

# Save to CSV
data.to_csv(output_file_path, index=False)

print(f'Data saved to {output_file_path}')


# 🗺️ Numerical Precision with Polygon Grid Data

## Purpose
Precision plays a key role in geoprocessing tasks, particularly when calculating areas from polygon grids in GIS. In this example, we calculate the area of each cell in a polygon grid system. While this method is often used in geospatial analyses, it may not be as precise as methods based on structured grid systems. Understanding the limitations of precision in such operations is crucial for accurate modeling and analysis.

## Examples
Here are some common operations involving float variables and geospatial data:

- **Calculate the area of each polygon in a shapefile**: This shows how spatial polygons from a grid system can be used to compute areas.
- **Ensure valid geometries and correct CRS**: Ensuring that the geometries are valid and in the correct coordinate system is crucial for accuracy.
- **Convert areas to specific units**: The area can be converted to other units (e.g., hectares) as needed.

By mastering these geoprocessing techniques and understanding how precision influences polygon-based grid systems, you can improve the accuracy and reliability of spatial analyses in GIS workflows.


# 🤖 Example Query

<div class="alert alert-block alert-info">
<b>GPT Text:</b>

- **Objective**: Calculate the grid cell size and area grid shapefile that has a single polygon for each grid element.
- **Data**: grid.shp polygon grid element
- **Output Format**: Save the output in a CSV file named `gridarea.csv` grid size and area.
- **Dependencies**: Use GeoPandas to handle data reading, calculations, and writing to CSV.
- **Path**: `C:\Users\Karen\Chat GPT Workshop\Data\Grids`
- **Request**: Can you create a Python script to calculate grid size and area based polygons in the shapefile then write this data to a CSV file?
</div>

## ▶️ Code Block

The following code block is the result of this query. (Insert your Python code here if needed)

In [ ]:
import geopandas as gpd

# File paths
input_file_path = base_path / 'Data' / 'Grids' / 'grid.shp'
output_file_path = base_path / 'Data' / 'Grids' / 'gridarea.csv'

# Load the shapefile
gdf = gpd.read_file(input_file_path)

# Calculate the area of each polygon (assumed to be in CRS units that represent meters or feet)
gdf['area'] = gdf['geometry'].area

# Assuming a square grid, calculate the grid size as the square root of the area
gdf['grid_size'] = gdf['area'] ** 0.5

# Round grid size and area to the nearest integer
gdf['grid_size'] = gdf['grid_size'].round().astype(int)
gdf['area'] = gdf['area'].round().astype(int)

# Save to CSV
gdf[['grid_size', 'area']].to_csv(output_file_path, index=False)

print(f'Data saved to {output_file_path}')

